# Урок 1. Введение, примеры задач, бизнес- и ML-метрики

Домашнее задание
1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

ROC AUC - Насколько высока концентрация интересных товаров в начале списка рекомендаций.

Cumulative gain at K (CG@K) — базовая метрика ранжирования, которая использует простую идею: чем релевантные элементы в этом топе, тем лучше

Discounted cumulative gain at K (DCG@K) — модификация cumulative gain at K, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции

Mean reciprocal rank (MRR) 

Spearman Correlation - Корреляция (Спирмена) реального и прогнозируемого рангов рекомендаций.

Коэффициент корреляции Кендэлла, который основан на подсчете согласованных
(и несогласованных) пар у перестановок — пар элементов, котором перестановки присвоили одинаковый (разный) порядок.

PFound. Разработка Яндекс, которая используется для оценки качества ранжирования.

2) Доделать все функции, где стоит комментарий "сделать дома"

In [1]:
import numpy as np

recommended_list = [143, 523, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [143, 523, 1134, 991]

prices_recommended = [400, 60, 40, 40 , 90]
prices_bought = [400, 60, 40, 40 , 90]

In [2]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

def hit_rate_at_k(recommended_list, bought_list, k=5):
    
#   сделать в домашней работе

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [3]:
hit_rate(recommended_list, bought_list)

1

In [4]:
hit_rate_at_k (recommended_list, bought_list)

1

In [5]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
#     сделать дома
    # Лучше считать через скалярное произведение, а не цикл
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [6]:
precision(recommended_list, bought_list)

0.4

In [7]:
precision_at_k(recommended_list, bought_list, k=5)

0.8

In [8]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.8571428571428571

In [9]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

def recall_at_k(recommended_list, bought_list, k=5):
    
    # сделать дома
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)
    
    return recall



def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    # сделать дома
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)

    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]

    flags = np.isin(recommended_list, bought_list)

    recall = (prices_recommended * flags).sum() / prices_bought.sum()
    
    return recall

In [10]:
recall(recommended_list, bought_list)

1.0

In [11]:
recall_at_k(recommended_list,bought_list,k=5)

1.0

In [12]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.8571428571428571

In [13]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [14]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
  

In [15]:
ap_k(recommended_list, bought_list, k=5)

1.0

In [16]:
def map_k(recommended_list, bought_list, k=5):
    
    # сделать дома
    s = len(bought_list)
    list_ap_k = [ap_k(recommended_list[i], bought_list[i], k) for i in np.arange(s)]
    print(list_ap_k)
    result = sum(list_ap_k)/s
    
    return result

In [17]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

[1.0, 0, 0]


0.3333333333333333

In [18]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    
    # сделать дома
    recommended_list = recommended_list[:k]
    ranks = 0
    for item_rec in recommended_list:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    result = ranks / len(recommended_list)
    return result

In [19]:
reciprocal_rank(recommended_list, bought_list, k=5)

0.41666666666666663